In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train', 'test', 'extraimages', 'sample_submission_file.csv']


In [2]:
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
import tensorflow as tf
from tensorflow.python.framework import ops
import math
import glob
# from skimage.transform import resize   # for resizing images

In [3]:
#reading image values in pixels width * hight * channels
ext = ['jpg', 'jpeg']    # Add image formats here
data = []
labels = []

files = []
imdir = '../input/train/train/cbb/'
[files.extend(glob.glob(imdir + '*.' + e)) for e in ext]
data.extend([cv2.imread(file) for file in files])
labels.extend(["cbb" for file in files])

files = []
imdir = '../input/train/train/cbsd/'
[files.extend(glob.glob(imdir + '*.' + e)) for e in ext]
data = np.concatenate([data, [cv2.imread(file) for file in files]])
labels.extend(["cbsd" for file in files])

files = []
imdir = '../input/train/train/cgm/'
[files.extend(glob.glob(imdir + '*.' + e)) for e in ext]
data = np.concatenate([data, [cv2.imread(file) for file in files]])
labels.extend(["cgm" for file in files])


files = []
imdir = '../input/train/train/cmd/'
[files.extend(glob.glob(imdir + '*.' + e)) for e in ext]
data = np.concatenate([data, [cv2.imread(file) for file in files]])
labels.extend(["cmd" for file in files])

files = []
imdir = '../input/train/train/healthy/'
[files.extend(glob.glob(imdir + '*.' + e)) for e in ext]
data = np.concatenate([data, [cv2.imread(file) for file in files]])
labels.extend(["healthy" for file in files])


In [4]:
size = (300, 300)
data = np.array([cv2.resize(d, size, interpolation = cv2.INTER_AREA) for d in data])

In [5]:
# data = np.concatenate([data, [np.fliplr(data[i]) for i in range(len(data))]])
# labels.extend([labels[i] for i in range(len(labels))])

In [6]:
labels = np.array(labels)

In [7]:
lr = [np.fliplr(data[i]) for i in range(len(data))]
labels_lr = [labels[i] for i in range(len(labels))]

In [8]:
data = np.concatenate([data, lr])
labels = np.concatenate([labels, labels_lr])

In [9]:
# data = np.array(data)
labels = np.array(labels)

In [10]:
labels = pd.get_dummies(labels)

In [11]:
X_train = data
Y_train = labels

In [12]:
# #using only training data
# X_train, X_test, Y_train, Y_test = train_test_split(data, labels, test_size=.05, random_state=42, stratify=labels)

In [13]:
#using real test data
testData = []
files = []
imdir = '../input/test/test/0/'
[files.extend(glob.glob(imdir + '*.' + e)) for e in ext]
testData.extend([cv2.imread(file) for file in files])

size = (300, 300)
testData = [cv2.resize(d, size, interpolation = cv2.INTER_AREA) for d in testData]

X_train = data
Y_train = labels
X_test = np.array(testData)
testLabels = files

In [14]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline

Using TensorFlow backend.


In [15]:
input_shape = ((300, 300, 3))
X_input = Input(input_shape)
    
# Zero-Padding: pads the border of X_input with zeroes
X = ZeroPadding2D((3, 3))(X_input)

# CONV -> BN -> RELU Block applied to X
X = Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0')(X)
X = BatchNormalization(axis = 3, name = 'bn0')(X)
X = Activation('relu')(X)

# MAXPOOL
X = AveragePooling2D((2, 2), name='avg_pool0')(X)

# CONV -> BN -> RELU Block applied to X
X = Conv2D(32, (5, 5), strides = (1, 1), name = 'conv1')(X)
X = BatchNormalization(axis = 3, name = 'bn1')(X)
X = Activation('relu')(X)

# MAXPOOL
X = AveragePooling2D((2, 2), name='avg_pool1')(X)

# CONV -> BN -> RELU Block applied to X
X = Conv2D(32, (3, 3), strides = (1, 1), name = 'conv2')(X)
X = BatchNormalization(axis = 3, name = 'bn2')(X)
X = Activation('relu')(X)

# MAXPOOL
X = AveragePooling2D((2, 2), name='avg_pool2')(X)

# FLATTEN X (means convert it to a vector) + FULLYCONNECTED
X = Flatten()(X)
X = Dense(1024, activation="relu")(X)
X = Dropout(0.5)(X)
X = Dense(5, activation='softmax', name='fc')(X)

# Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
model = Model(inputs = X_input, outputs = X, name='satellite')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 300, 300, 3)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 306, 306, 3)       0         
_________________________________________________________________
conv0 (Conv2D)               (None, 300, 300, 32)      4736      
_________________________________________________________________
bn0 (BatchNormalization)     (None, 300, 300, 32)      128       
_________________________________________________________________
activation_1 (Activation)    (None, 300, 300, 32)      0         
_________________________________________________________________
avg_pool0 (AveragePooling2D) (None, 150, 150, 32)      0         
_________________________________________________________________
conv1 (Conv2D)               (None, 146, 146, 32)      25632     
__________

In [17]:
model.compile(optimizer="sgd", loss="mean_squared_error", metrics=["accuracy"])

In [18]:
model.fit(x=X_train, y=Y_train, epochs=50, batch_size=32)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
11312/11312 [==============================] - 36s 3ms/step - loss: 0.1317 - acc: 0.5290
Epoch 2/50
11312/11312 [==============================] - 32s 3ms/step - loss: 0.1118 - acc: 0.6001
Epoch 3/50
11312/11312 [==============================] - 32s 3ms/step - loss: 0.1042 - acc: 0.6292
Epoch 4/50
11312/11312 [==============================] - 32s 3ms/step - loss: 0.0983 - acc: 0.6528
Epoch 5/50
11312/11312 [==============================] - 32s 3ms/step - loss: 0.0933 - acc: 0.6708
Epoch 6/50
11312/11312 [==============================] - 32s 3ms/step - loss: 0.0882 - acc: 0.6887
Epoch 7/50
11312/11312 [==============================] - 32s 3ms/step - loss: 0.0846 - acc: 0.7016
Epoch 8/50
11312/11312 [==============================] - 32s 3ms/step - loss: 0.0804 - acc: 0.7174
Epoch 9/50
11312/11312 [==============================] - 32s 3ms/step - loss: 0.0759 - acc: 0.7403
Epoch 10/50
11312/11312 [===========================

In [19]:
# preds = model.evaluate(x=X_test, y=Y_test)
# print()
# print ("Loss = " + str(preds[0]))
# print ("Test Accuracy = " + str(preds[1]))

In [20]:
pred = model.predict(x=X_test)

In [21]:
pred = [np.argmax(pred[i]) for i in range(len(pred))]

In [22]:
for i in range(len(pred)):
    if pred[i] == 0:
        pred[i] = 'cbb'
    if pred[i] == 1:
        pred[i] = 'cbsd'
    if pred[i] == 2:
        pred[i] = 'cgm'
    if pred[i] == 3:
        pred[i] = 'cmd'
    if pred[i] == 4:
        pred[i] = 'healthy'

In [23]:
testLabels = [l[21:] for l in testLabels]

In [24]:
submission = pd.DataFrame(list(zip(pred, testLabels)), columns=["Category", "Id"])

In [25]:
submission.to_csv('submission.csv', index=False)

In [26]:
# import the modules we'll need
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "data.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

# create a random sample dataframe
df = pd.DataFrame(np.random.randn(50, 4), columns=list('ABCD'))

# create a link to download the dataframe
create_download_link(submission)